# Tensorboard

In [ ]:
import tensorflow as tf
import numpy as np
tf.compat.v1.reset_default_graph()
x = np.linspace(0, 2*np.pi, 100)
y = np.cos(x)
summary_writer = tf.summary.create_file_writer('tensorboard_log')
with summary_writer.as_default():
    for i, j in enumerate(y):
        tf.summary.scalar('cos', j, step=i)

In [ ]:
import tensorflow as tf
import datetime
import time
tf.compat.v1.reset_default_graph()
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax') ])
start_time = time.time()
model = create_model()
# tf.data.Dataset.from_tensor_slices：這可以將 Python 的 List、Dict、NumPy 陣列，包裝成 Dataset 物件。
# Dataset 物件#  #  1.Create a source dataset from your input data.
#  2.Apply dataset transformations to preprocess the data.3#  Iterate over the dataset and process the elements.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')
@tf.function
def train_step(x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss)
    train_accuracy(y_train, predictions)
@tf.function
def test_step(x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)
    test_loss(loss)
    test_accuracy(y_test, predictions)
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
EPOCHS = 5
for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    for (x_test, y_test) in test_dataset:
        test_step(x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
print("-----{} seconds -------------".format(time.time()-start_time))

# Tensorboard (keras)

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="fit_logs/", histogram_freq=1)
model.fit(x=x_train,y=y_train,
          epochs=5,validation_data=(x_test, y_test),callbacks=[tensorboard_callback])

In [ ]:
# 補充 :
# 1 引入相關套件
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# 2.使用 from_tensor_slices 函數，自陣列建立 Dataset
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
# 3 使用for迴圈可自 Dataset 取出所有資料 
for elem in dataset:
    print(elem.numpy())
print('*'*20)    
# 4 使用 Iterator/Next 每次取一個
# iterator
it = iter(dataset)
# 一次取一筆
print(next(it).numpy())
print(next(it).numpy())

# 5.求總和
gfg = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5]) 
# state：目前狀態(累計結果)。
# value：元素值。
# 0: axis 0 : dataset.reduce(axis, lambda state, value: state + value)
print(gfg.reduce(0, lambda x, y: x + y).numpy()) 
print('6 *'*20)
# 6 Map
gfg = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5]) 

for elem in gfg.map(lambda x: x * 2):
    print(elem.numpy())
print('7 *'*20)    
# 7 篩選(Filter)
# Filter
gfg = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5]) 

filter = lambda x: x % 2 == 0
for elem in gfg.filter(filter):
    print(elem.numpy())

In [ ]:
def count(stop):
    i = 0
    while i<stop:
        yield i
        i += 1

# args=[25] ： 最多從 count 函數取25個    
ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types=tf.int32, output_shapes = (), )
# repeat()：取完會重頭再開始
# batch(10)：一次取 10 個
# take(5)：取 5 次
for count_batch in ds_counter.repeat().batch(10).take(5):
    print(count_batch.numpy())
    
#  每批取不定個數。  
def gen_series():
    i = 0
    while True:
        size = np.random.randint(0, 10)
        yield i, np.random.normal(size=(size,))
        i += 1
        
for i, series in gen_series():
    print(i, ":", str(series))
    if i > 5:
        break    
        
# 洗牌(shuffle)。  
# 一次取 20 個洗牌，取完，再抽 20 個洗牌
# batch(10)：一次取 10 個
# take(5)：取 5 次
for count_batch in ds_counter.shuffle(20).batch(10).take(5):
    print(count_batch.numpy())
    
# 下載檔案
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

# rescale：特徵縮放
# rotation_range：自動增補旋轉20度內的圖片
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

images, labels = next(img_gen.flow_from_directory(flowers))

import matplotlib.pyplot as plt
plt.imshow(images[0])
labels[0]    

# 從目錄取得影像。
ds = tf.data.Dataset.from_generator(
    img_gen.flow_from_directory, args=[flowers], 
    output_types=(tf.float32, tf.float32), 
    output_shapes=([32,256,256,3], [32,5])
)

fsns_test_file = tf.keras.utils.get_file("fsns.tfrec", "https://storage.googleapis.com/download.tensorflow.org/data/fsns-20160927/testdata/fsns-00000-of-00001")

dataset = tf.data.TFRecordDataset(filenames = [fsns_test_file])

raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())
parsed.features.feature['image/text']
)

# 讀取三個檔案directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']

file_paths = [
    tf.keras.utils.get_file(file_name, directory_url + file_name)
    for file_name in file_names
]

# 合併多個檔案為一資料集
dataset = tf.data.TextLineDataset(file_paths)
# 讀取5行資料
for line in dataset.take(5):
    print(line.numpy())
    
# 結合 DataFrame。    
titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
df = pd.read_csv(titanic_file, index_col=None)
df.head()

titanic_slices = tf.data.Dataset.from_tensor_slices(dict(df))

for feature_batch in titanic_slices.take(1):
    for key, value in feature_batch.items():
        print("  {!r:20s}: {}".format(key, value))
        
# 選擇部份欄位     
titanic_batches = tf.data.experimental.make_csv_dataset(
    titanic_file, batch_size=4,
    label_name="survived", select_columns=['class', 'fare', 'survived'])

# 顯示欄位值    
for feature_batch, label_batch in titanic_batches.take(1):
    print("'survived': {}".format(label_batch))
    print("features:")
    for key, value in feature_batch.items():
        print("    {!r:20s}: {}".format(key, value))    